In [1]:
from objects.instance import Instance
from script.data import loader 

In [2]:
data = loader('la01')
inst = Instance(**data)

In [3]:
# Recherche des taches possibles à t
t=0
def recherche_tach(inst,t):
    tache = []
    for j in inst.jobs_list:
        if j.current_task.machine.state=="Free":
            tache.append(j.current_task)
    
    # prio
    # Recherche du max
    for k in range(len(tache)):
        maxi = -1
        for e in tache:
            if e.duration > maxi:
                if e.machine.state == "Free":
                    maxi = e.duration
                    maxi_task = e
        
        maxi_task.update_task_and_date("On going",t)
        maxi_task.machine.update_current_task(maxi_task)
        maxi_task.job.update_all_task()
        try:
            tache.remove(maxi_task)
        except:
            pass
    for j in inst.jobs_list:
        j.update_all_task()

In [4]:
recherche_tach(inst,t)

In [5]:
for res in inst.resource_list:
    print(res)

Machine number = 0, State = Busy, Current Job = 4, Current Task = 0
Machine number = 1, State = Busy, Current Job = 3, Current Task = 0
Machine number = 2, State = Busy, Current Job = 7, Current Task = 0
Machine number = 3, State = Busy, Current Job = 6, Current Task = 0
Machine number = 4, State = Busy, Current Job = 9, Current Task = 0


In [6]:
for i in range(37,40):
    for res in inst.resource_list:
        if res.current_task.finishDate < i:
            print("true")
            print(res.current_job)
            res.state= "Free"
            res.current_job.update_current_task(res.current_task.taskID+1)
            res.current_task.update_task("Done")
            res.current_job.current_task.update_task("On going")
            print(res.current_job)

            recherche_tach(inst,i)

true
Task list
Task # 0 : JobID= 7, taskID= 0, Start= 0, Finish=38, Machine= Machine number = 2, State = Busy, Current Job = 7, Current Task = 0, State= On going
Task # 1 : JobID= 7, taskID= 1, Start= 0, Finish=60, Machine= Machine number = 0, State = Busy, Current Job = 4, Current Task = 0, State= Not Started
Task # 2 : JobID= 7, taskID= 2, Start= 0, Finish=41, Machine= Machine number = 1, State = Busy, Current Job = 3, Current Task = 0, State= Not Started
Task # 3 : JobID= 7, taskID= 3, Start= 0, Finish=24, Machine= Machine number = 3, State = Busy, Current Job = 6, Current Task = 0, State= Not Started
Task # 4 : JobID= 7, taskID= 4, Start= 0, Finish=83, Machine= Machine number = 4, State = Busy, Current Job = 9, Current Task = 0, State= Not Started
Job state= Not Done
Task list
Task # 0 : JobID= 7, taskID= 0, Start= 0, Finish=38, Machine= Machine number = 2, State = Free, Current Job = 7, Current Task = 0, State= Done
Task # 1 : JobID= 7, taskID= 1, Start= 0, Finish=60, Machine= Mac

In [7]:
for re in inst.resource_list:
    print(re)

Machine number = 0, State = Busy, Current Job = 4, Current Task = 0
Machine number = 1, State = Busy, Current Job = 3, Current Task = 0
Machine number = 2, State = Free, Current Job = 7, Current Task = 0
Machine number = 3, State = Busy, Current Job = 6, Current Task = 0
Machine number = 4, State = Busy, Current Job = 9, Current Task = 0


In [60]:
    
    # sélectionner l'opération (i,j) de plus grande priorité  




NameError: name 'separate' is not defined